# DL Brain project

In [1]:
%ls /home/arasdar/datasets/DL-BrainBody/Control/

O10test1.txt   O2valid3.txt  O4valid2.txt  O6valid1.txt  O8train.txt
O10test2.txt   O3test1.txt   O4valid3.txt  O6valid2.txt  O8valid1.txt
O10test3.txt   O3test2.txt   O5test1.txt   O6valid3.txt  O8valid2.txt
O10train.txt   O3test3.txt   O5test2.txt   O7test1.txt   O8valid3.txt
O10valid1.txt  O3train.txt   O5test3.txt   O7test2.txt   O9test1.txt
O10valid2.txt  O3valid1.txt  O5train.txt   O7test3.txt   O9test2.txt
O10valid3.txt  O3valid2.txt  O5valid1.txt  O7train.txt   O9test3.txt
O2test1.txt    O3valid3.txt  O5valid2.txt  O7valid1.txt  O9train.txt
O2test2.txt    O4test1.txt   O5valid3.txt  O7valid2.txt  O9valid1.txt
O2test3.txt    O4test2.txt   O6test1.txt   O7valid3.txt  O9valid2.txt
O2train.txt    O4test3.txt   O6test2.txt   O8test1.txt   O9valid3.txt
O2valid1.txt   O4train.txt   O6test3.txt   O8test2.txt
O2valid2.txt   O4valid1.txt  O6train.txt   O8test3.txt


In [46]:
%ls /home/arasdar/datasets/DL-BrainBody/PD/

P10test1.txt   P11valid3.txt  P4valid2.txt  P6valid1.txt  P8train.txt
P10test2.txt   P1test1.txt    P4valid3.txt  P6valid2.txt  P8valid1.txt
P10test3.txt   P1test2.txt    P5test1.txt   P6valid3.txt  P8valid2.txt
P10train.txt   P1test3.txt    P5test2.txt   P7test1.txt   P8valid3.txt
P10valid1.txt  P1train.txt    P5test3.txt   P7test2.txt   P9test1.txt
P10valid2.txt  P1valid1.txt   P5train.txt   P7test3.txt   P9test2.txt
P10valid3.txt  P1valid2.txt   P5valid1.txt  P7train.txt   P9test3.txt
P11test1.txt   P1valid3.txt   P5valid2.txt  P7valid1.txt  P9train.txt
P11test2.txt   P4test1.txt    P5valid3.txt  P7valid2.txt  P9valid1.txt
P11test3.txt   P4test2.txt    P6test1.txt   P7valid3.txt  P9valid2.txt
P11train.txt   P4test3.txt    P6test2.txt   P8test1.txt   P9valid3.txt
P11valid1.txt  P4train.txt    P6test3.txt   P8test2.txt
P11valid2.txt  P4valid1.txt   P6train.txt   P8test3.txt


In [47]:
import numpy as np
import pandas as pd

In [75]:
X_02train = pd.read_csv(filepath_or_buffer='/home/arasdar/datasets/DL-BrainBody/Control/O2train.txt', header=None, 
                        delim_whitespace=True)
X_03train = pd.read_csv(filepath_or_buffer='/home/arasdar/datasets/DL-BrainBody/Control/O3train.txt', header=None, 
                        delim_whitespace=True)
X_04train = pd.read_csv(filepath_or_buffer='/home/arasdar/datasets/DL-BrainBody/Control/O4train.txt', header=None, 
                        delim_whitespace=True)
X_10train = pd.read_csv(filepath_or_buffer='/home/arasdar/datasets/DL-BrainBody/PD/P10train.txt', header=None, 
                        delim_whitespace=True)
X_11train = pd.read_csv(filepath_or_buffer='/home/arasdar/datasets/DL-BrainBody/PD/P11train.txt', header=None, 
                        delim_whitespace=True)

In [76]:
Xtrain = np.array([X_02train.values, X_03train.values, X_04train.values, X_10train.values, X_11train.values])
Ytrain = np.array([               0,                0,                0,                1,                1])
# Ytrain = np.array([Controllllllllll, Controllllllllll, Controllllllllll, PDDDDDDDDDDDDDDD, PDDDDDDDDDDDDD])

In [77]:
Xtrain.shape, Ytrain.shape, Xtrain.dtype, Ytrain.dtype

((5, 81920, 56), (5,), dtype('float64'), dtype('int64'))

In [78]:
Xtrain.reshape([5, 1, -1, 56]).shape, Ytrain.shape

((5, 1, 81920, 56), (5,))

In [79]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


In [80]:
# five subjects, 8 activities, 10 seconds each, sr=256 samples/sec
# 256 sampling rate: 256 samples per second
Xtrain.shape, \
Xtrain.reshape(5, 40, 8, 256, 56).shape, \
Xtrain.reshape([5*40, 8, 256, 56]).shape

((5, 81920, 56), (5, 40, 8, 256, 56), (200, 8, 256, 56))

In [81]:
(Ytrain.reshape([-1, 1]) * np.ones([5, 40])).shape

(5, 40)

In [82]:
(Ytrain.reshape([-1, 1]) * np.ones([5, 40])).reshape([-1]).shape

(200,)

In [83]:
Xtrain = Xtrain.reshape([200, 8, 256, 56])
Ytrain = (Ytrain.reshape([-1, 1]) * np.ones([5, 40])).reshape([-1])

In [84]:
inputs = Xtrain.copy() # inputs
indices = Ytrain.copy() # outputs

In [85]:
inputs.shape, indices.shape

((200, 8, 256, 56), (200,))

In [145]:
# input_size = Xtrain[0].shape
batch_size = 5 # num subjects
seq_length = 40 # rnn input sequence: this is dynamic/ not static
input_size = (8, 256, 56) # one second of each activity which is 1024=256sample/sec*4sec
output_size = 2 # num classes
hidden_size = 56*2*2*2*2*2
learning_rate = 1e-4

In [146]:
def model_input(input_size, hidden_size, batch_size):
    # CNN and MLP
    inputs = tf.placeholder(dtype=tf.float32, shape=[200, *input_size], name='inputs')
    indices = tf.placeholder(dtype=tf.int32, shape=[200], name='indices')    
    training = tf.placeholder(dtype=tf.bool, shape=[], name='training') # batchnorm
    # RNN
    cell = tf.nn.rnn_cell.GRUCell(num_units=hidden_size)
    #cell = tf.nn.rnn_cell.LSTMCell(hidden_size)
    cells = tf.nn.rnn_cell.MultiRNNCell(cells=[cell], state_is_tuple=True)
    initial_state = cells.zero_state(batch_size, tf.float32) # ??? None/batch size
    return inputs, indices, training, cells, initial_state

In [147]:
model_input(input_size=input_size, hidden_size=hidden_size, batch_size=batch_size)

(<tf.Tensor 'inputs_1:0' shape=(200, 8, 256, 56) dtype=float32>,
 <tf.Tensor 'indices_1:0' shape=(200,) dtype=int32>,
 <tf.Tensor 'training_1:0' shape=() dtype=bool>,
 (<tf.Tensor 'MultiRNNCellZeroState_1/GRUCellZeroState/zeros:0' shape=(5, 1792) dtype=float32>,))

In [178]:
def discriminator(inputs, input_size, output_size, # CNN
                  batch_size, initial_state, cells, # RNN/MLP/FC
                  alpha=0.1, training=False, reuse=False): # NL/BN
    with tf.variable_scope('discriminator', reuse=reuse):
        # CNN
        # NHWC format
        #?, 8, 256, 56
        print(inputs.shape)
        h1 = tf.layers.conv2d(inputs=inputs, filters=64, kernel_size=[8, 3], strides=[1, 2], padding='same')
        bn1 = tf.layers.batch_normalization(h1, training=training)
        nl1 = tf.maximum(alpha * bn1, bn1)
        print(nl1.shape)
        h2 = tf.layers.conv2d(inputs=nl1, filters=128, kernel_size=[8, 3], strides=[1, 2], padding='same')
        bn2 = tf.layers.batch_normalization(h2, training=training)
        nl2 = tf.maximum(alpha * bn2, bn2)
        print(nl2.shape)
        h3 = tf.layers.conv2d(inputs=nl2, filters=256, kernel_size=[8, 3], strides=[1, 2], padding='same')
        bn3 = tf.layers.batch_normalization(h3, training=training)
        nl3 = tf.maximum(alpha * bn3, bn3)
        print(nl3.shape)
        h4 = tf.layers.conv2d(inputs=nl3, filters=512, kernel_size=[8, 3], strides=[1, 2], padding='same')
        bn4 = tf.layers.batch_normalization(h4, training=training)
        nl4 = tf.maximum(alpha * bn4, bn4)
        print(nl4.shape)
        h5 = tf.layers.conv2d(inputs=nl4, filters=1024, kernel_size=[8, 3], strides=[1, 2], padding='same')        
        bn5 = tf.layers.batch_normalization(h5, training=training)
        nl5 = tf.maximum(alpha * bn5, bn5)
        print(nl5.shape)
        
        ###################### flattenning
        h5 = tf.layers.conv2d(inputs=nl5, filters=1024, kernel_size=[8, 3], strides=[2, 2], padding='same')        
        bn5 = tf.layers.batch_normalization(h5, training=training)
        nl5 = tf.maximum(alpha * bn5, bn5)
        print(nl5.shape)
        h5 = tf.layers.conv2d(inputs=nl5, filters=1024, kernel_size=[8, 3], strides=[2, 2], padding='same')        
        bn5 = tf.layers.batch_normalization(h5, training=training)
        nl5 = tf.maximum(alpha * bn5, bn5)
        print(nl5.shape)
        h5 = tf.layers.conv2d(inputs=nl5, filters=1024, kernel_size=[8, 3], strides=[2, 2], padding='same')        
        bn5 = tf.layers.batch_normalization(h5, training=training)
        nl5 = tf.maximum(alpha * bn5, bn5)
        print(nl5.shape)

        # RNN/MLP/Fully Connected
        inputs_rnn = tf.reshape(tensor=nl5, shape=[batch_size, -1, 1024]) # ?, 40, VEC
        print('inputs_rnn.shape, initial_state[0].shape', 
              inputs_rnn.shape, initial_state[0].shape) # tuple=True
        # uni-directional/bi-directional: dynamic/static on sequence length
        outputs_rnn, final_state = tf.nn.dynamic_rnn(cell=cells, inputs=inputs_rnn, initial_state=initial_state)
        print('outputs_rnn.shape, final_state[0].shape',
              outputs_rnn.shape, final_state[0].shape) # tuple=True
        outputs = tf.reshape(outputs_rnn, [-1, 56*2*2*2*2*2]) # ?x40, VEC
        print('outputs.shape', outputs.shape)

        # Output layer
        logits = tf.layers.dense(inputs=outputs, units=output_size)
        print(logits.shape)
        return logits, final_state

In [187]:
def model_loss(inputs, indices, input_size, output_size, hidden_size, batch_size, cells, initial_state, training):
    logits, final_state = discriminator(inputs=inputs, input_size=input_size, output_size=output_size, 
                                        cells=cells, initial_state=initial_state, batch_size=batch_size, 
                                        training=training)
    labels = tf.one_hot(indices=indices, depth=output_size, dtype=logits.dtype)
    print(logits.shape, labels.shape)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    # If argmax label and argmax logits are equal: true/false
    if_equal_bool = tf.equal(x=tf.argmax(axis=1, input=logits), y=tf.argmax(axis=1, input=labels))
    accuracy = tf.reduce_mean(axis=0, input_tensor=tf.cast(dtype=tf.float32, x=if_equal_bool))
    return loss, accuracy

In [188]:
def model_opt(loss, learning_rate):
    tvars = tf.trainable_variables()
    gvars = [var for var in tvars if var.name.startswith('discriminator')]
    # gvars = []
    # for var in tvars: 
    #     if var.name.startswith('discriminator'):
    #         gvars.append(var)

    # Optimize MLP/CNN
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        #opt = tf.train.AdamOptimizer(learning_rate).minimize(loss, var_list=g_vars) # CNN
        #grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(loss, g_vars), clip_norm=5) # usually around 1-5
        grads = tf.gradients(loss, gvars) # RNN
        opt = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads_and_vars=zip(grads, gvars))
    return opt

In [189]:
class Model:
    def __init__(self, input_size, output_size, hidden_size, batch_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.inputs, self.indices, self.training, cells, self.initial_state = model_input(input_size=input_size, 
                                                                                          batch_size=batch_size, 
                                                                                          hidden_size=hidden_size)
        # Create the Model: calculating the loss and forwad pass
        self.loss, self.acc = model_loss(inputs=self.inputs, indices=self.indices, batch_size=batch_size, 
                                         input_size=input_size, output_size=output_size, hidden_size=hidden_size,
                                         cells=cells, initial_state=self.initial_state, training=self.training)

        # Update the model: backward pass and backprop
        self.opt = model_opt(loss=self.loss, learning_rate=learning_rate)

In [190]:
# def get_batches(X, Y, batch_size):
#     """ Return a generator for batches """
#     n_batches = len(X) // batch_size
#     X, Y = X[:n_batches*batch_size], Y[:n_batches*batch_size]

#     # Loop over batches and yield
#     for b in range(0, len(X), batch_size):
#         yield X[b:b+batch_size], Y[b:b+batch_size]

In [191]:
graph = tf.reset_default_graph()

model = Model(input_size=input_size, hidden_size=hidden_size, output_size=output_size, 
              learning_rate=learning_rate, batch_size=batch_size)

(200, 8, 256, 56)
(200, 8, 128, 64)
(200, 8, 64, 128)
(200, 8, 32, 256)
(200, 8, 16, 512)
(200, 8, 8, 1024)
(200, 4, 4, 1024)
(200, 2, 2, 1024)
(200, 1, 1, 1024)
inputs_rnn.shape, initial_state[0].shape (5, 40, 1024) (5, 1792)
outputs_rnn.shape, final_state[0].shape (5, 40, 1792) (5, 1792)
outputs.shape (200, 1792)
(200, 2)
(200, 2) (200, 2)


In [192]:
model.inputs.shape, inputs.shape

(TensorShape([Dimension(200), Dimension(8), Dimension(256), Dimension(56)]),
 (200, 8, 256, 56))

In [193]:
model.indices.shape, indices.shape

(TensorShape([Dimension(200)]), (200,))

In [194]:
saver = tf.train.Saver() 
train_loss, train_acc = [], []

with tf.Session(graph=graph) as sess:
    sess.run(fetches=tf.global_variables_initializer())
    #saver.restore(sess,'checkpoints/model.ckpt')
    initial_state = sess.run(model.initial_state)
    
    # Epoch/episode
    for epoch in range(1):
        #for Xinputs, Yindices in get_batches(X=Xtrain, Y=Ytrain, batch_size=batch_size):
        loss, acc, _ = sess.run(fetches=[model.loss, model.acc, model.opt], 
                                feed_dict={model.inputs: inputs, 
                                           model.indices: indices, 
                                           model.initial_state: initial_state,
                                           model.training: True})
        print('loss:{}'.format(loss))
        print('accuracy:{}'.format(acc))
        train_loss.append(loss)
        train_acc.append(acc)
    
    # Saving the trained and validated model
    saver.save(sess,'checkpoints/model.ckpt')

loss:0.762806236743927
accuracy:0.4050000011920929
